# Odhad CDPF

## Cíl analýzy

Cílem analýzy je odhad neoklasické agregátní CDPF.

## Data a metoda

Pro odhad byly použity následující hodnoty:

- hrubá přidaná hodnota (Gross Value Added) (převedená na stálé ceny roku 2010 pomocí deflátoru HDP),
- čistý fixní nefinanční kapitál (převedený na stálé ceny roku 2010 pomocí deflátoru kapitálových zásob),
- celkový počet odpracovaných hodin.

In [2]:
import eurostat
import pandas
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller
from scipy import stats
import statsmodels.tsa.stattools as ts
import numpy
import seaborn

LOCATIONS = ["CZ", "AT", "EU27_2020"]
BASE_LOCATION = "CZ"
START_YEAR = 1980
END_YEAR = 2020

### Hrubá přidaná hodnota (Gross Value Added)

Hrubá přidaná hodnota je načtena na základě kódu `nama_10_a10`.

In [19]:
data_1_es = eurostat.get_data_df('nama_10_a10')

MultiIndex([('geo\time',    ''),
            (    'year',    ''),
            ( 'CP_MNAC', 'B1G'),
            ( 'CP_MNAC',  'D1'),
            ('PD10_NAC', 'B1G')],
           names=['unit', 'na_item'])

In [65]:
data_1 = data_1_es
# Value added, gross
data_1 = data_1[data_1["na_item"].isin(["B1G", "D1"])]
# Total, all NACE activities
data_1 = data_1[data_1["nace_r2"] == "TOTAL"]
# Current prices, million euro
data_1 = data_1[data_1["unit"].isin(["CP_MNAC", "PD10_NAC"])]
data_1 = pandas.melt(data_1, id_vars=["unit", "na_item", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)),
                     value_name="value", var_name="year")
data_1 = data_1.pivot(index=["geo\\time", "year"], columns=["unit", "na_item"], values="value")
data_1 = data_1.dropna()
data_1[("CP_MNAC", "B1G")] = data_1[("CP_MNAC", "B1G")] * data_1[("PD10_NAC", "B1G")] / 100
data_1[("CP_MNAC", "D1")] = data_1[("CP_MNAC", "D1")] * data_1[("PD10_NAC", "B1G")] / 100
data_1 = data_1.drop([("PD10_NAC", "B1G")], axis=1)
data_1.columns = ['_'.join(col) for col in data_1.columns.values]
data_1 = data_1.rename({"CP_MNAC_B1G": "gross_value_added", "CP_MNAC_D1": "compensation_of_employees"}, axis=1)
data_1 = data_1.reset_index()
data_1.head()

,geo\time,year,gross_value_added,compensation_of_employees
0,AT,1995,130145.540670,73793.930850
1,AT,1996,134886.309636,75360.493182
2,AT,1997,140135.537164,77354.111597
3,AT,1998,146283.001040,80443.033918
4,AT,1999,151754.564940,83621.655376


Pro data provedeme logaritmizaci a určíme hodnoty diferencí.

In [ ]:
data_1 = data_1.sort_values(["geo\\time", "year"])
data_1["ln_gross_value_added"] = numpy.log(data_1["gross_value_added"])
data_1["ln_gross_value_added_t_minus_1"] = data_1.groupby("geo\\time")["ln_gross_value_added"].shift(1)
data_1["ln_gross_value_added_diff"] = (data_1["ln_gross_value_added"] - data_1["ln_gross_value_added_t_minus_1"]) / data_1["ln_gross_value_added"]
data_1 = data_1[["geo\\time", "year", "ln_gross_value_added", "ln_gross_value_added_diff"]]
data_1.tail()


Jako předposlední krok je načten čistý fixní nefinanční kapitál, který má kód `nama_10_nfa_bs`.

In [ ]:
data_3_es = eurostat.get_data_df('nama_10_nfa_bs')

data_3 = data_3_es
data_3 = eurostat.get_data_df('nama_10_nfa_bs')
# Total fixed assets (net)
data_3 = data_3[data_3["asset10"] == "N11N"]
# Total, all activities
data_3 = data_3[data_3["sector"] == "S1"]
# Current fixed prices, millions of national currency
data_3 = data_3[data_3["unit"] == "CP_MNAC"]
data_3 = pandas.melt(data_3, id_vars=["unit", "asset10", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)),
                    value_name="total_fixed_assets", var_name="year")
data_3 = data_3.sort_values(["geo\\time", "year"])
data_3 = data_3.dropna()

V posledním kroku je načten deflátor kapitálových zásob, který má kód `nama_10_nfa_fl`.

In [ ]:
data_4_es = eurostat.get_data_df('nama_10_nfa_fl')

data_4 = data_4_es
data_4 = data_4[data_4["asset10"] == "N11G"]
data_4 = data_4[data_4["nace_r2"] == "TOTAL"]
data_4 = data_4[data_4["unit"] == "PD10_NAC"]
data_4 = pandas.melt(data_4, id_vars=["unit", "asset10", "geo\\time"], value_vars=list(range(START_YEAR, END_YEAR + 1)),
                    value_name="capital_stock_deflator", var_name="year")
data_4 = data_4.sort_values(["geo\\time", "year"])
data_4 = data_4.dropna()

Hodnota kapitálu je nyní převedena na stálé ceny roku 2010.

In [ ]:
data_3 = data_3.merge(data_4, on=["geo\\time", "year"])
data_3["total_fixed_assets"] = data_3["total_fixed_assets"] * data_3["capital_stock_deflator"]
data_3["ln_total_fixed_assets"] = numpy.log(data_3["total_fixed_assets"])
data_3["ln_total_fixed_assets_t_minus_1"] = data_3.groupby("geo\\time")["ln_total_fixed_assets"].shift(1)
data_3["ln_total_fixed_assets_diff"] = (data_3["ln_total_fixed_assets"] - data_3["ln_total_fixed_assets_t_minus_1"]) / data_3["ln_total_fixed_assets"]
data_3 = data_3[["geo\\time", "year", "ln_total_fixed_assets", "ln_total_fixed_assets_diff"]]